<a href="https://colab.research.google.com/github/Himasri17/FMML/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.Yes, averaging validation accuracy across multiple splits typically provides more consistent and reliable results. This approach is commonly used in cross-validation techniques, such as k-fold cross-validation, where the dataset is divided into multiple folds, and the model is trained and validated on different subsets of the data.

### Benefits of Averaging Validation Accuracy Across Multiple Splits

1. **Reduced Variance**: Averaging over multiple splits reduces the variance caused by random splits of the data. Single splits can be biased if the training and validation sets are not representative of the overall data distribution.

2. **Better Generalization Estimate**: Averaging provides a more robust estimate of how the model will perform on unseen data, as it considers different portions of the data in both training and validation.

3. **Mitigates the Effect of Outliers**: Single splits might be significantly affected by outliers or particular data points. Multiple splits dilute the influence of such anomalies, leading to more stable accuracy measurements.

4. **Consistency and Reliability**: Averaging smoothens the results, making them less sensitive to the specific data partition used in each split, thus providing a clearer picture of the model’s performance.

5. **Improved Hyperparameter Tuning**: When used during hyperparameter tuning, averaged metrics across multiple splits provide more reliable feedback, helping in better model optimization.
2.Yes, averaging validation accuracy across multiple splits typically provides a more accurate estimate of test accuracy. This is because it addresses several key factors that contribute to the reliability of performance metrics:

### How Averaging Improves Test Accuracy Estimates

1. **Better Representation of Data Variability**: By using multiple splits, the model is exposed to various subsets of the data, including different combinations of training and validation samples. This exposure helps the model learn patterns that are more representative of the overall dataset, leading to a better generalization estimate.

2. **Minimizes Overfitting to Specific Splits**: Single splits can lead to models that perform well on a particular validation set but not necessarily on unseen data. Averaging over multiple splits reduces the likelihood that the model's performance is artificially inflated due to the peculiarities of a single split.

3. **Captures a Wider Range of Scenarios**: Each split tests the model on different portions of the data, ensuring that performance isn't overly influenced by any one segment. This approach captures a broader range of possible scenarios that the model might encounter in real-world testing.

4. **Reduces the Impact of Randomness**: The randomness inherent in data splitting can cause significant fluctuations in performance metrics. Averaging mitigates this randomness, leading to a more stable and reliable estimate of how the model will perform on new, unseen data.

5. **More Reliable Indicator of True Performance**: Since test accuracy is ultimately about how well the model generalizes, using multiple validation splits provides a closer approximation of this by effectively simulating multiple independent test scenarios.

### Cross-Validation as a Common Approach

The most common approach for achieving this is k-fold cross-validation, where the data is divided into k subsets, and the model is trained and validated k times, each time using a different subset as the validation set and the rest for training. The average of these validation results serves as a robust proxy for test accuracy.

### Key Takeaway

Averaging validation accuracy across multiple splits does not just provide consistency—it also serves as a more accurate indicator of a model’s expected performance on unseen test data, making it an essential practice for robust model evaluation.
3.The number of iterations (or folds, in the case of cross-validation) significantly affects the accuracy and stability of the performance estimate. Generally, increasing the number of iterations provides a more precise and reliable estimate of test accuracy, but there are trade-offs to consider.

### Effects of Increasing the Number of Iterations

1. **Improved Accuracy of the Estimate**:
   - More iterations mean the model is trained and validated on more combinations of the data. This reduces the impact of any single split that might be unrepresentative, leading to a more accurate estimate of how the model will perform on unseen data.

2. **Reduced Variance in Estimates**:
   - A higher number of iterations smoothens the fluctuations in accuracy that can result from random variations in data splits. This reduction in variance makes the estimate more reliable and less sensitive to particular data partitions.

3. **Better Use of Data**:
   - With more iterations, especially in k-fold cross-validation, each data point is used multiple times for training and validation. For instance, in 10-fold cross-validation, each data point serves as a validation point once and as a training point nine times, maximizing the utility of the dataset.

4. **Lower Bias in Estimates**:
   - A single split or a small number of iterations might not capture the complete data distribution, leading to biased estimates. More iterations help average out these biases, providing a closer reflection of true model performance.

### Trade-offs of Higher Iterations

1. **Increased Computational Cost**:
   - Each additional iteration requires training and evaluating the model again, which can significantly increase the overall computation time, especially with large datasets or complex models.

2. **Diminishing Returns**:
   - While increasing the number of iterations improves estimate quality, the marginal gain diminishes after a certain point. For example, moving from 5-fold to 10-fold cross-validation usually provides a noticeable improvement, but increasing from 10-fold to 20-fold yields less pronounced gains relative to the additional computational effort.

3. **Potential Overfitting in Some Cases**:
   - Excessive iterations might lead to overfitting on the validation process itself if hyperparameters are overly tuned based on these repeated evaluations, although this is more of a concern in hyperparameter tuning than in basic performance estimation.

### Optimal Number of Iterations

- **Common Practice**: A typical choice is 5 or 10 iterations (folds), as this strikes a balance between computational efficiency and the reliability of the performance estimate.
- **High-Variance Data**: For highly variable or small datasets, more iterations (e.g., 10-20 folds) can be beneficial to obtain a robust estimate.
- **Time Constraints**: When computational resources or time are limited, a smaller number of iterations (e.g., 5 folds) may be used, accepting slightly less precision.

### Key Takeaway

Increasing the number of iterations generally leads to a better estimate of test accuracy by reducing variance and bias. However, the improvements plateau after a point, and computational costs must be balanced against the gains in estimation accuracy.
4.Increasing the number of iterations can help mitigate the effects of having a very small training or validation dataset, but it does not entirely solve the limitations associated with small datasets. Here’s how increased iterations impact small datasets and the extent to which they can help:

### How Increased Iterations Affect Small Datasets

1. **Maximizes Use of Data**:
   - With more iterations, each data point gets used multiple times in both training and validation sets. For example, in k-fold cross-validation, every point is used as part of the training set in \(k-1\) iterations and as part of the validation set in 1 iteration. This maximization of data usage is particularly beneficial when the dataset is small.

2. **Reduces Bias and Variance**:
   - For small datasets, individual splits might not capture the overall data distribution, leading to high bias or variance in the estimates. By averaging across many iterations, the model's performance is assessed on diverse subsets, providing a more reliable estimate of its generalization capability.

3. **Lessens the Impact of Outliers or Anomalies**:
   - In small datasets, the effect of outliers or anomalies is more pronounced in individual splits. Increasing the number of iterations helps average out the influence of these unusual points, stabilizing the accuracy estimates.

4. **Provides Better Insight into Model Performance**:
   - With limited data, each training-validation cycle becomes critical in understanding how the model might generalize. More iterations allow for a more nuanced view of performance, revealing how well the model handles variability in training data.

### Limitations of Increasing Iterations on Small Datasets

1. **Diminishing Returns**:
   - Simply increasing iterations does not add new data; it only reshuffles existing data. While this can provide more robust estimates, it cannot fully compensate for the fundamental limitations of having insufficient data for learning complex patterns.

2. **Risk of Overfitting to Small Validation Sets**:
   - In very small datasets, each validation set might be tiny, leading to high variance in performance estimates. Even with many iterations, the small validation size can make it challenging to get a reliable measure of how the model performs on unseen data.

3. **Poor Generalization**:
   - Small training sets can lead to models that underfit or overfit due to the lack of sufficient data for capturing underlying trends. Even with increased iterations, the model might not learn adequately if the training samples are too few or unrepresentative.

4. **High Variance in Small Validation Scores**:
   - When validation sets are very small, performance scores can fluctuate significantly between iterations. Averaging many such scores can still result in an unstable or unreliable estimate, especially if certain validation sets do not represent the broader data distribution well.

### Practical Approaches for Small Datasets

- **Data Augmentation**: In fields like image recognition, augmenting data (e.g., rotating, flipping images) can increase the effective size of the training set, helping the model learn better despite limited original data.
  
- **Cross-Validation with Stratification**: Ensuring that each split maintains the overall distribution of classes (in classification tasks) helps to make each fold more representative.

- **Leave-One-Out Cross-Validation (LOOCV)**: For very small datasets, LOOCV (where each sample is used as a validation set once) makes maximum use of data. However, it can be computationally expensive and may still suffer from high variance.

- **Simpler Models or Regularization**: Using simpler models or adding regularization can prevent overfitting when training data is limited.

### Key Takeaway

Increasing the number of iterations helps extract as much information as possible from small datasets and provides more stable performance estimates. However, it cannot overcome the fundamental limitations of having a very small amount of data, and careful attention to model choice, regularization, and data augmentation remains essential.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

The performance of k-Nearest Neighbors (k-NN) classifiers, such as 3-NN and 1-NN, can be sensitive to the number of splits and the size of each split during cross-validation. Here’s a detailed look at how these factors affect their accuracy:

### Effect of the Number of Splits on k-NN Classifiers

1. **3-NN Classifier**:
   - **More Splits (Higher k-fold)**: As the number of splits increases (e.g., from 5 to 10 or even LOOCV), each fold uses a larger proportion of the dataset for training and a smaller proportion for validation. This generally provides a more reliable estimate of the model’s performance by reducing the variability of accuracy scores across folds. The 3-NN classifier, which considers the three nearest neighbors, often benefits from this as it reduces the effect of noise or outliers in validation sets.
   - **Stability and Robustness**: With more splits, the averaged performance across folds becomes more stable, as each validation set is smaller and provides a finer test of generalization.

2. **1-NN Classifier**:
   - **More Splits (Higher k-fold)**: 1-NN is particularly sensitive to the individual data points since it bases its prediction solely on the nearest neighbor. More splits can lead to high variability in performance, especially in small validation sets where a single point can dramatically influence the result. However, with more iterations, the average accuracy provides a broader view of performance across multiple partitions, potentially smoothing out some fluctuations.
   - **Higher Variance**: Since 1-NN relies on just one nearest point, it tends to have higher variance in accuracy compared to 3-NN. Increasing the number of splits can help mitigate this variance slightly by averaging over many iterations, but it does not eliminate the inherent sensitivity of the 1-NN classifier.

### Effect of Split Size on k-NN Classifiers

1. **3-NN Classifier**:
   - **Larger Training Sets (Smaller Validation Sets)**: As the split size increases (i.e., more training data per split), the 3-NN classifier generally performs better because it has more data to accurately define local structures and identify the nearest neighbors. This leads to better generalization and smoother decision boundaries.
   - **Smaller Training Sets (Larger Validation Sets)**: When the training set is small, the 3-NN classifier can suffer from reduced accuracy because fewer data points are available to define the neighborhood, increasing the likelihood of noisy or unrepresentative neighbors affecting predictions.

2. **1-NN Classifier**:
   - **Larger Training Sets (Smaller Validation Sets)**: 1-NN benefits from larger training sets because it has more data points to choose from when finding the nearest neighbor, which helps reduce the impact of noise and anomalies. However, the 1-NN classifier is still prone to higher variance because each decision is highly localized to a single point.
   - **Smaller Training Sets (Larger Validation Sets)**: With fewer training points, the 1-NN classifier can struggle as it may not have a representative point for certain regions, leading to erratic or overly sensitive predictions.

### Comparative Results: 3-NN vs. 1-NN

1. **Accuracy Stability**:
   - **3-NN** generally provides more stable accuracy estimates across different splits because it averages decisions across three neighbors, making it less sensitive to individual data points.
   - **1-NN** is less stable, with accuracy varying more significantly depending on the split due to its reliance on a single neighbor for each decision.

2. **Sensitivity to Split Size**:
   - **3-NN** performs better with larger training sizes and can manage small validation sets more effectively due to its averaging effect.
   - **1-NN** is more affected by the split size and can have extreme performance variations, especially with smaller training sizes.

3. **Bias-Variance Trade-off**:
   - **3-NN** has slightly higher bias but lower variance compared to 1-NN. This makes it generally more robust, especially when split sizes are not optimal.
   - **1-NN** has low bias but high variance, which can lead to high accuracy in some splits but poor generalization overall.

### Key Takeaway

- **3-NN** tends to have a more stable and reliable accuracy across varying numbers of splits and split sizes, making it preferable in most scenarios, especially when noise or outliers are present.
- **1-NN** is more sensitive to split sizes and the number of splits, often resulting in higher variance and less consistent performance. Increasing the number of splits helps average out some of this variance but does not fully address the sensitivity of 1-NN to individual data points.

Ultimately, 3-NN provides a more balanced performance, especially when evaluated across multiple cross-validation folds, making it generally more suitable for practical use compared to 1-NN, particularly in noisy or small dataset scenarios.